In [1]:
from util import load_bestmoves, load_mblist
from tqdm import tqdm

def is_strongly_solved(ai):
    bestmoves = load_bestmoves()
    mblist = load_mblist()
    for mb in tqdm(mblist):
        if set(ai(mb, candidate=True)) > set(bestmoves[tuple(mb.records)]):
            return False
    return True

In [2]:
from ai import ai14s

is_strongly_solved(ai14s)

100%|██████████| 294778/294778 [00:49<00:00, 5914.87it/s]


True

In [3]:
class Check_solved:
    bestmoves = load_bestmoves()
    mblist = load_mblist()
    
    @staticmethod
    def is_strongly_solved(ai):
        count = 0
        for mb in tqdm(Check_solved.mblist):
            if set(ai(mb, candidate=True)) <= set(Check_solved.bestmoves[tuple(mb.records)]):
                count += 1
        nodenum = len(Check_solved.mblist)
        print(f"{count}/{nodenum} {count/nodenum*100:.2f}%")
        return count == nodenum

In [4]:
Check_solved.is_strongly_solved(ai14s)

100%|██████████| 294778/294778 [00:51<00:00, 5721.66it/s]

294690/294778 99.97%


False

In [5]:
a = set([1, 2])
b = set([1, 2, 3])
print(a <= b)
print(a > b)

True
False


In [6]:
print(b <= a)
print(b > a)

False
True


In [7]:
a = set([1, 2])
b = set([2, 3])
print(a <= b)
print(a > b)

False
False


In [8]:
@staticmethod
def is_strongly_solved(ai):
    count = 0
    incorrectlist = []
    for mb in tqdm(Check_solved.mblist):
        candidate = set(ai(mb, candidate=True))
        bestmoves = set(Check_solved.bestmoves[tuple(mb.records)])
        if candidate <= bestmoves:
            count += 1
        else:
            incorrectlist.append((mb, candidate, bestmoves))
    nodenum = len(Check_solved.mblist)
    print(f"{count}/{nodenum} {count/nodenum*100:.2f}%")
    return count == nodenum, incorrectlist

Check_solved.is_strongly_solved = is_strongly_solved

In [9]:
result, incorrectlist = Check_solved.is_strongly_solved(ai14s)

100%|██████████| 294778/294778 [00:50<00:00, 5833.35it/s]

294690/294778 99.97%


In [10]:
mb, candidate, bestmoves = incorrectlist[0]
print(mb)
print("candidate", candidate)
print("bestmoves", bestmoves)

Turn o
o.X
...
...

candidate {(1, 1)}
bestmoves {(0, 1), (0, 2), (2, 2)}


In [11]:
print(incorrectlist[0])

(<marubatsu.Marubatsu object at 0x0000020B2AE5D0D0>, {(1, 1)}, {(0, 1), (0, 2), (2, 2)})


In [12]:
from marubatsu import Marubatsu

mb = Marubatsu()
ai14s(mb, candidate=True)

[(1, 1)]

In [13]:
print(mb.board)

[['.', '.', '.'], ['.', '.', '.'], ['.', '.', '.']]


In [14]:
print(tuple(mb.board))

(['.', '.', '.'], ['.', '.', '.'], ['.', '.', '.'])


In [15]:
coords = ['A1', 'A2', 'B1', 'B2', 'C1']
print("".join(coords))

A1A2B1B2C1


In [16]:
txt = ""
for col in mb.board:
    txt += "".join(col)
print(txt)

.........


In [17]:
txt = "".join(["".join(col) for col in mb.board])
print(txt)

.........


In [18]:
def board_to_str(self):
    txt = ""
    for col in self.board:
        txt += "".join(col)
    return txt

Marubatsu.board_to_str = board_to_str

In [19]:
print(mb.board_to_str())
mb.move(0, 0)
print(mb.board_to_str())
mb.move(1, 0)
print(mb.board_to_str())
mb.move(0, 2)
print(mb.board_to_str())
mb.move(2, 2)
print(mb.board_to_str())

.........
o........
o..x.....
o.ox.....
o.ox....x


In [22]:
from tree import Mbtree

mbtree = Mbtree.load("../data/aidata")
bestmoves_by_board = {}
for node in tqdm(mbtree.nodelist):
    txt = node.mb.board_to_str()
    if not txt in bestmoves_by_board.keys():
        bestmoves_by_board[txt] = node.bestmoves

100%|██████████| 549946/549946 [00:01<00:00, 521190.05it/s]


In [23]:
len(bestmoves_by_board)

5478

In [24]:
import pickle
import gzip
with gzip.open("../data/bestmoves_by_board.dat", "wb") as f:
    pickle.dump(bestmoves_by_board, f)

In [25]:
from random import choice

def ai_gt6(mb, bestmoves_by_board, candidate=False):
    if candidate:
        return bestmoves_by_board[mb.board_to_str()]
    else:
        return choice(bestmoves_by_board[mb.board_to_str()])

In [26]:
from ai import ai2s, ai_match

ai_match(ai=[ai_gt6, ai2s], params=[{"bestmoves_by_board": bestmoves_by_board}, {}])

ai_gt6 VS ai2s


100%|██████████| 10000/10000 [00:30<00:00, 322.85it/s]

count     win    lose    draw
o        9683       0     317
x        7780       0    2220
total   17463       0    2537

ratio     win    lose    draw
o       96.8%    0.0%    3.2%
x       77.8%    0.0%   22.2%
total   87.3%    0.0%   12.7%



In [27]:
@staticmethod
def is_strongly_solved(ai, params=None):
    if params is None:
        params = {}
    count = 0
    incorrectlist = []
    for mb in tqdm(Check_solved.mblist):
        candidate = set(ai(mb, candidate=True, **params))
        bestmoves = set(Check_solved.bestmoves[tuple(mb.records)])
        if candidate <= bestmoves:
            count += 1
        else:
            incorrectlist.append((mb, candidate, bestmoves))
    nodenum = len(Check_solved.mblist)
    print(f"{count}/{nodenum} {count/nodenum*100:.2f}%")
    return count == nodenum, incorrectlist
    
Check_solved.is_strongly_solved = is_strongly_solved

In [28]:
params = {"bestmoves_by_board": bestmoves_by_board}
result, incorrectlist = Check_solved.is_strongly_solved(ai_gt6, params=params)
print(result)

100%|██████████| 294778/294778 [00:01<00:00, 270543.00it/s]

294778/294778 100.00%
True


In [29]:
bestmoves_by_board = {}
mblist_by_board = []
for node in mbtree.nodelist:
    txt = node.mb.board_to_str()
    if not txt in bestmoves_by_board.keys():
        bestmoves_by_board[txt] = node.bestmoves
        if node.mb.status == Marubatsu.PLAYING:
            mblist_by_board.append(node.mb)

In [30]:
len(mblist_by_board)

4520

In [31]:
with gzip.open("../data/mblist_by_board.dat", "wb") as f:
    pickle.dump(mblist_by_board, f)

In [32]:
class Check_solved:
    bestmoves_by_board = load_bestmoves("../data/bestmoves_by_board.dat")
    mblist_by_board = load_mblist("../data/mblist_by_board.dat")
    
    @staticmethod
    def is_strongly_solved(ai, params=None):
        if params is None:
            params = {}
        count = 0
        incorrectlist = []
        for mb in tqdm(Check_solved.mblist_by_board):
            candidate = set(ai(mb, candidate=True, **params))
            bestmoves = set(Check_solved.bestmoves_by_board[mb.board_to_str()])
            if candidate <= bestmoves:
                count += 1
            else:
                incorrectlist.append((mb, candidate, bestmoves))
        nodenum = len(Check_solved.mblist_by_board)
        print(f"{count}/{nodenum} {count/nodenum*100:.2f}%")
        return count == nodenum, incorrectlist

In [33]:
result, incorrectlist = Check_solved.is_strongly_solved(ai14s)
print(result)

100%|██████████| 4520/4520 [00:01<00:00, 2639.10it/s]

4468/4520 98.85%
False


In [34]:
params = {"bestmoves_by_board": bestmoves_by_board}
result, incorrectlist = Check_solved.is_strongly_solved(ai_gt6, params=params)
print(result)

100%|██████████| 4520/4520 [00:00<00:00, 107608.52it/s]

4520/4520 100.00%
True


In [35]:
class Check_solved:
    bestmoves_by_board = None
    mblist_by_board = None
    
    @staticmethod
    def is_strongly_solved(ai, params=None):
        if Check_solved.bestmoves_by_board is None:
            Check_solved.bestmoves_by_board = load_bestmoves ("../data/bestmoves_by_board.dat")
        if Check_solved.mblist_by_board is None:
            Check_solved.mblist_by_board = load_mblist("../data/mblist_by_board.dat")            
        if params is None:
            params = {}
        count = 0
        incorrectlist = []
        for mb in tqdm(Check_solved.mblist_by_board):
            candidate = set(ai(mb, candidate=True, **params))
            bestmoves = set(Check_solved.bestmoves_by_board[mb.board_to_str()])
            if candidate <= bestmoves:
                count += 1
            else:
                incorrectlist.append((mb, candidate, bestmoves))
        nodenum = len(Check_solved.mblist_by_board)
        print(f"{count}/{nodenum} {count/nodenum*100:.2f}%")
        return count == nodenum, incorrectlist

In [37]:
result, incorrectlist = Check_solved.is_strongly_solved(ai14s)
print(result)
params = {"bestmoves_by_board": bestmoves_by_board}
result, incorrectlist = Check_solved.is_strongly_solved(ai_gt6, params=params)
print(result)

100%|██████████| 4520/4520 [00:01<00:00, 2644.79it/s]


4468/4520 98.85%
False


100%|██████████| 4520/4520 [00:00<00:00, 110638.59it/s]

4520/4520 100.00%
True
